In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Lib

Finding geographical coordinates for Atlanta, GA USA

In [2]:
address = 'Atlanta, GA'

geolocator = Nominatim(user_agent="atl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Atlanta are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Atlanta are 33.7490987, -84.3901849.


Creating the links to download the csv files I found on the Internet for this project, specifically zip code and neighborhood info

In [3]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import os

filepath ='https://docs.google.com/spreadsheets/d/e/2PACX-1vSZaqlui-2xMDWWeYkzqOaxFhdheFsGUpTZV8MUkjX7vWaaGda3t2VI8t7B9HohBkYivizJQVsFN-od/pub?gid=674492145&single=true&output=csv'
filepath2 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQeflARQnXsBdO-Orrrj61liwG9yP_UCU3jGPozeigSzUiOv5sqofoxphwIcgfPjpZBnInz7FjjKjdS/pub?gid=781352977&single=true&output=csv'

In [4]:
df = pd.read_csv(filepath)
df = df.filter(['Zip','Neighborhood'], axis=1)
df

,Zip,Neighborhood
0,30002,Avondale Estates
1,30021,Clarkston
2,30030,City of Decatur/Oakhurst/Winnona Park
3,30032,East Lake / South Decatur
4,30033,North Decatur
5,30080,Smyrna
6,30084,Tucker
7,30303,Downtown / Central Business District / Fairle...
8,30305,Buckhead / Tuxedo Park / Garden Hills / Hayne...
9,30306,Virginia Highlands / Morningside / Lenox Park...


Looking at zip codes in the entire state of Georgia to ensure I'm capturing the relevant zip codes in the metro Atlanta area

In [5]:
df2 = pd.read_csv(filepath2)
df2.head()

,Zip,Latitude,Longitude
0,31150,33.844371,-84.474050
1,31646,30.518903,-82.080630
2,31826,32.803518,-84.693840
3,30334,33.702657,-84.439127
4,31606,30.802411,-83.196240


In [6]:
atlanta = pd.merge(left=df, right=df2, left_on='Zip', right_on='Zip')
atlanta

,Zip,Neighborhood,Latitude,Longitude
0,30002,Avondale Estates,33.772122,-84.26491
1,30021,Clarkston,33.809604,-84.23870
2,30030,City of Decatur/Oakhurst/Winnona Park,33.771705,-84.29407
3,30032,East Lake / South Decatur,33.740055,-84.26791
4,30033,North Decatur,33.811354,-84.28335
5,30080,Smyrna,33.875780,-84.50382
6,30084,Tucker,33.853853,-84.22036
7,30303,Downtown / Central Business District / Fairle...,33.752856,-84.39013
8,30305,Buckhead / Tuxedo Park / Garden Hills / Hayne...,33.830054,-84.38472
9,30306,Virginia Highlands / Morningside / Lenox Park...,33.786755,-84.35149


Mapping Atlanta zip codes on city area map

In [7]:
map_atlanta = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(atlanta['Latitude'],atlanta['Longitude'], atlanta['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_atlanta)  
    
map_atlanta

In [8]:
CLIENT_ID = 'RM0IXCNGIMPPZ14RBHT0QMC4V20CNPUMQRMDWNC0X4PDH023' # your Foursquare ID
CLIENT_SECRET = 'FDXRQG4E3YVFLUVWJLJ12RVKBWD50LGHSYODHXXGUMOPZ0AG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RM0IXCNGIMPPZ14RBHT0QMC4V20CNPUMQRMDWNC0X4PDH023
CLIENT_SECRET:FDXRQG4E3YVFLUVWJLJ12RVKBWD50LGHSYODHXXGUMOPZ0AG


Imported my 4sq info and ensured that I could read from the API by checking the first neighborhood on the list

In [9]:
neighborhood_latitude = atlanta.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = atlanta.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = atlanta.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of  Avondale Estates are 33.772121999999996, -84.26491.


In [10]:
LIMIT = 400 # limit of number of venues returned by Foursquare API



radius = 5000 # define radius



# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display UR


'https://api.foursquare.com/v2/venues/explore?&client_id=RM0IXCNGIMPPZ14RBHT0QMC4V20CNPUMQRMDWNC0X4PDH023&client_secret=FDXRQG4E3YVFLUVWJLJ12RVKBWD50LGHSYODHXXGUMOPZ0AG&v=20180605&ll=33.772121999999996,-84.26491&radius=5000&limit=400'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9a3c146d8c56001b90bb92'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Avondale Estates',
  'headerFullLocation': 'Avondale Estates',
  'headerLocationGranularity': 'city',
  'totalResults': 216,
  'suggestedBounds': {'ne': {'lat': 33.81712204500004,
    'lng': -84.21087601234815},
   'sw': {'lat': 33.72712195499995, 'lng': -84.31894398765185}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56f6a924498e9eff48634ecf',
       'name': 'Rising Son',
       'location': {'address': '124 N Avondale Rd',
        'lat': 33.77641685720219,
        'lng': -84.2675154031931,
        'labeledLatLngs': [{'label':

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Rising Son,Restaurant,33.776417,-84.267515
1,My Parents' Basement,Beer Bar,33.775612,-84.272334
2,Arepa Mia,Venezuelan Restaurant,33.776942,-84.267327
3,Lifeline Animal Project,Dog Run,33.776895,-84.269454
4,Good Karma Coffee House,Gluten-free Restaurant,33.776037,-84.265889


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:

atl_venues = getNearbyVenues(names=atlanta['Neighborhood'],
                                   latitudes=atlanta['Latitude'],
                                   longitudes=atlanta['Longitude']
                                  )



 Avondale Estates
 Clarkston
 City of Decatur/Oakhurst/Winnona Park
 East Lake / South Decatur
 North Decatur
 Smyrna
 Tucker 
 Downtown / Central Business District / Fairlee Poplar
 Buckhead / Tuxedo Park / Garden Hills / Haynes Manor / Peachtree Battle / Peachtree Hills
 Virginia Highlands / Morningside / Lenox Park / Poncey Highland / Druid Hills
 Candler Park / Druid Hills / Edgewood / Emory / Inman Park / Lake Claire / Little Five Points
 Midtown / Old Fourth Ward
 Midtown / Ansley Park / Brookwood Hills / Loring Heights
 Adair Park / Capitol View / Oakland City / West End
 Cascade
 Downtown Atlanta / Grant Park
 Downtown Atlanta / Castleberry Hills
 Vines City / Mozeley Park
 Grant Park peoplestown / Lakewood
 Cabbagetown / East Atlanta Village /  Ormewood Park /South Dekalb
 East Lake / Kirkwood / Edgewood
 Home Park /Northwest Atlanta / Collier Hills / Underwood hills / Midtown West
 Brookhaven / North Atlanta / Dunwoody
 Morningside/Lenox Park / Piedmont Heights / Lenox / Lavi

The code above was created to ensure I could get venue coordinates and types for future analysis.  Specifically, I ensured I could download the JSON file from 4sq, look at venues a certain distance from the zip code's coordinates, and categorize each entry.  Below you'll see this output:

In [16]:
atl_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Avondale Estates,33.772122,-84.26491,Rising Son,33.776417,-84.267515,Restaurant
1,Avondale Estates,33.772122,-84.26491,Good Karma Coffee House,33.776037,-84.265889,Gluten-free Restaurant
2,Avondale Estates,33.772122,-84.26491,Avondale Lake & Bess Walking Park,33.768739,-84.266118,Park
3,Avondale Estates,33.772122,-84.26491,Arepa Mia,33.776942,-84.267327,Venezuelan Restaurant
4,Avondale Estates,33.772122,-84.26491,Avondale Estates,33.775358,-84.267100,City


I'm now proceeding to normalize the data to determine the top venue categories in each neighborhood

In [17]:
# one hot encoding
atl_onehot = pd.get_dummies(atl_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
atl_onehot['Neighborhood'] = atl_venues['Neighborhood'] 

cols = list(atl_onehot.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('Neighborhood')) #Remove Nrighborhood from list
atl_onehot = atl_onehot[['Neighborhood'] + cols] #Create new dataframe with columns in the order you want
atl_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,City,Clothing Store,Cocktail Bar,Coffee Shop,College & University,College Basketball Court,College Bookstore,College Rec Center,College Theater,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taxi,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai R

In [18]:
atl_grouped = atl_onehot.groupby('Neighborhood').mean().reset_index()
atl_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,City,Clothing Store,Cocktail Bar,Coffee Shop,College & University,College Basketball Court,College Bookstore,College Rec Center,College Theater,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taxi,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai R

I'm now finding the top 20 venue types in each neighborhood to find any interesting insights to guide my business decision.  Specifically, I'm looking at places where one of the top destinations is a type of Asian restaurant and where it isn't to determine where my opportunities could lie and to keep in mind for further in the exercise.  I selected Asian restaurant after seeing the frequency and variety of them throughout the Atlanta database (sushi, vietnamese, Bubble Tea, Indian, etc.)

In [24]:
num_top_venues = 20

for hood in atl_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = atl_grouped[atl_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adair Park / Capitol View / Oakland City / West End----
                              venue  freq
0                    Discount Store  0.07
1                       Gas Station  0.06
2                              Park  0.05
3     Vegetarian / Vegan Restaurant  0.05
4               Fried Chicken Joint  0.05
5                       Pizza Place  0.05
6                             Trail  0.04
7                    Cosmetics Shop  0.04
8                              Bank  0.04
9                    Sandwich Place  0.04
10                       Donut Shop  0.04
11                Convenience Store  0.02
12               Seafood Restaurant  0.02
13                    Grocery Store  0.02
14                     Concert Hall  0.02
15             Fast Food Restaurant  0.02
16             Caribbean Restaurant  0.02
17               Chinese Restaurant  0.02
18                      Wings Joint  0.02
19  Southern / Soul Food Restaurant  0.02


---- Avondale Estates----
                     venue  f

Beginning the clustering exercise to segment the neighborhoods and find similarities.

In [32]:
# set number of clusters
kclusters = 5

atl_grouped_clustering = atl_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(atl_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 4, 1, 1, 1, 1, 0, 3, 0], dtype=int32)

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = atl_grouped['Neighborhood']

for ind in np.arange(atl_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(atl_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adair Park / Capitol View / Oakland City / We...,Gas Station,Vegetarian / Vegan Restaurant,Park,Discount Store,Fried Chicken Joint,Pizza Place,Sandwich Place,Trail,Cosmetics Shop,Bank
1,Avondale Estates,Pizza Place,Bakery,Sandwich Place,Gas Station,Liquor Store,Brewery,Thrift / Vintage Store,Breakfast Spot,Pub,Pet Store
2,Briarcliff Woods / Oak Grove / Northlake,Pool,Coffee Shop,High School,Grocery Store,Motel,Nature Preserve,Chinese Restaurant,Gas Station,Shipping Store,Discount Store
3,Brookhaven / North Atlanta / Dunwoody,American Restaurant,Pizza Place,Sandwich Place,Mexican Restaurant,Gym / Fitness Center,Salon / Barbershop,Chinese Restaurant,Sports Bar,Restaurant,BBQ Joint
4,Buckhead / North Atlanta,Mexican Restaurant,American Restaurant,Grocery Store,Sporting Goods Shop,Coffee Shop,Shipping Store,Smoothie Shop,Gas Station,Jewelry Store,Bagel Shop
5,Buckhead / North Buckhead / Chastain Park / N...,Mexican Restaurant,Italian Restaurant,Park,Wings Joint,Bakery,Gym,Spa,Grocery Store,Burrito Place,Southern / Soul Food Restaurant
6,Buckhead / Tuxedo Park / Garden Hills / Hayne...,Steakhouse,American Restaurant,New American Restaurant,Pizza Place,Italian Restaurant,Gastropub,Grocery Store,Spa,Salon / Barbershop,Breakfast Spot
7,Cabbagetown / East Atlanta Village / Ormewoo...,Bar,Discount Store,Gas Station,Park,Pizza Place,Coffee Shop,Fast Food Restaurant,Video Store,Bank,Sushi Restaurant
8,Candler Park / Druid Hills / Edgewood / Emory...,Park,Pizza Place,Bar,BBQ Joint,Coffee Shop,Bookstore,Gift Shop,Southern / Soul Food Restaurant,Garden,Breakfast Spot
9,Cascade,Park,Gym / Fitness Center,Chinese Restaurant,Caribbean Restaurant,BBQ Joint,Golf Course,Seafood Restaurant,Nature Preserve,Sandwich Place,Discount Store


In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

atl_merged = atlanta

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
atl_merged = atl_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

atl_merged

,Zip,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,30002,Avondale Estates,33.772122,-84.26491,1,Pizza Place,Bakery,Sandwich Place,Gas Station,Liquor Store,Brewery,Thrift / Vintage Store,Breakfast Spot,Pub,Pet Store
1,30021,Clarkston,33.809604,-84.23870,0,Discount Store,Grocery Store,Sandwich Place,Fruit & Vegetable Store,Chinese Restaurant,Storage Facility,Kids Store,Seafood Restaurant,Shoe Store,Fried Chicken Joint
2,30030,City of Decatur/Oakhurst/Winnona Park,33.771705,-84.29407,2,Brewery,Park,Pub,Pizza Place,Italian Restaurant,Breakfast Spot,Dog Run,American Restaurant,Ice Cream Shop,Gastropub
3,30032,East Lake / South Decatur,33.740055,-84.26791,0,Discount Store,Wings Joint,Fast Food Restaurant,Grocery Store,Pizza Place,Gas Station,Seafood Restaurant,Fried Chicken Joint,Food,Video Store
4,30033,North Decatur,33.811354,-84.28335,0,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Fried Chicken Joint,Pizza Place,Liquor Store,Salon / Barbershop
5,30080,Smyrna,33.875780,-84.50382,3,Bar,Park,Pizza Place,Burger Joint,Hot Dog Joint,Burrito Place,Library,Fried Chicken Joint,Bookstore,Liquor Store
6,30084,Tucker,33.853853,-84.22036,0,Fast Food Restaurant,Wings Joint,Mexican Restaurant,Pizza Place,Burger Joint,Southern / Soul Food Restaurant,Ice Cream Shop,Bank,Pharmacy,Pet Store
7,30303,Downtown / Central Business District / Fairle...,33.752856,-84.39013,2,Park,Hotel,Art Gallery,Burger Joint,BBQ Joint,Fast Food Restaurant,Sports Club,Music Venue,Bar,Museum
8,30305,Buckhead / Tuxedo Park / Garden Hills / Hayne...,33.830054,-84.38472,1,Steakhouse,American Restaurant,New American Restaurant,Pizza Place,Italian Restaurant,Gastropub,Grocery Store,Spa,Salon / Barbershop,Breakfast Spot
9,30306,Virginia Highlands / Morningside / Lenox Park...,33.786755,-84.35149,3,Bar,Trail,American Restaurant,Pizza Place,Burger Joint,Italian Restaurant,Grocery Store,Pet Store,Dive Bar,BBQ Joint


In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(atl_merged['Latitude'], atl_merged['Longitude'], atl_merged['Neighborhood'], atl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Knowing how the neighborhoods are clustered, I now want to analyze each cluster and compare this info to the top 20 venues info from the code above to help me narrow my decision.

In [30]:
atl_merged.loc[atl_merged['Cluster Labels'] == 0, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Avondale Estates,Pizza Place,Brewery,Bakery,Gas Station,Thrift / Vintage Store,Sandwich Place,Breakfast Spot,Theater,Pub,Restaurant
1,Clarkston,Discount Store,Grocery Store,Sandwich Place,Fruit & Vegetable Store,Shoe Store,Intersection,Wings Joint,Chinese Restaurant,Storage Facility,Breakfast Spot
3,East Lake / South Decatur,Discount Store,Fast Food Restaurant,Wings Joint,Pizza Place,Grocery Store,Gas Station,Seafood Restaurant,American Restaurant,Video Store,Fried Chicken Joint
4,North Decatur,Indian Restaurant,Cosmetics Shop,Fast Food Restaurant,Discount Store,Grocery Store,American Restaurant,Sandwich Place,Salon / Barbershop,Restaurant,Fried Chicken Joint
6,Tucker,Wings Joint,Fast Food Restaurant,Mexican Restaurant,Cosmetics Shop,Discount Store,Mobile Phone Shop,Pharmacy,Southern / Soul Food Restaurant,Pet Store,Gift Shop
13,Adair Park / Capitol View / Oakland City / We...,Discount Store,Gas Station,Fried Chicken Joint,Pizza Place,Park,Vegetarian / Vegan Restaurant,Donut Shop,Trail,Bank,Sandwich Place
14,Cascade,Intersection,Gas Station,Liquor Store,Chinese Restaurant,Seafood Restaurant,Golf Course,BBQ Joint,Nature Preserve,Caribbean Restaurant,Dry Cleaner
17,Vines City / Mozeley Park,Gas Station,Park,Trail,Convenience Store,Pizza Place,Southern / Soul Food Restaurant,Caribbean Restaurant,Fried Chicken Joint,Discount Store,Sandwich Place
19,Cabbagetown / East Atlanta Village / Ormewoo...,Bar,Discount Store,Gas Station,Park,Pizza Place,Coffee Shop,Fast Food Restaurant,Nightclub,Gastropub,Ice Cream Shop
21,Home Park /Northwest Atlanta / Collier Hills ...,Gas Station,Furniture / Home Store,Performing Arts Venue,Park,Gym,Mobile Phone Shop,Fried Chicken Joint,Kitchen Supply Store,Liquor Store,Breakfast Spot


In [31]:
atl_merged.loc[atl_merged['Cluster Labels'] == 1, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Briarcliff Woods / Oak Grove / Northlake,Pool,Coffee Shop,High School,Intersection,Nature Preserve,Chinese Restaurant,Gas Station,Motel,Park,Sandwich Place


In [32]:
atl_merged.loc[atl_merged['Cluster Labels'] == 2, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,City of Decatur/Oakhurst/Winnona Park,Pub,Gastropub,Park,Brewery,Pizza Place,Ice Cream Shop,American Restaurant,Dog Run,Italian Restaurant,Cocktail Bar
5,Smyrna,Bar,Park,Burger Joint,Burrito Place,Latin American Restaurant,Library,Bookstore,Liquor Store,Fried Chicken Joint,Sushi Restaurant
7,Downtown / Central Business District / Fairle...,Hotel,Park,Hotel Bar,Coffee Shop,BBQ Joint,Art Gallery,Aquarium,Burger Joint,American Restaurant,Fast Food Restaurant
8,Buckhead / Tuxedo Park / Garden Hills / Hayne...,American Restaurant,Pizza Place,Steakhouse,New American Restaurant,Italian Restaurant,Salon / Barbershop,Coffee Shop,Furniture / Home Store,Spa,Gastropub
9,Virginia Highlands / Morningside / Lenox Park...,Bar,American Restaurant,Trail,Grocery Store,Italian Restaurant,Burger Joint,Pizza Place,Pub,Park,Gym
10,Candler Park / Druid Hills / Edgewood / Emory...,Bar,Pizza Place,Park,Movie Theater,Gift Shop,Coffee Shop,Bookstore,BBQ Joint,History Museum,Cuban Restaurant
11,Midtown / Old Fourth Ward,Southern / Soul Food Restaurant,Hotel,Coffee Shop,Trail,Grocery Store,American Restaurant,Park,New American Restaurant,Burger Joint,Mediterranean Restaurant
12,Midtown / Ansley Park / Brookwood Hills / Lor...,American Restaurant,Coffee Shop,Pizza Place,Hotel,Seafood Restaurant,Park,Café,Breakfast Spot,Performing Arts Venue,Mediterranean Restaurant
15,Downtown Atlanta / Grant Park,Coffee Shop,Pizza Place,Zoo Exhibit,American Restaurant,Breakfast Spot,Park,BBQ Joint,History Museum,Bar,Trail
16,Downtown Atlanta / Castleberry Hills,Hotel,Park,Coffee Shop,American Restaurant,Hotel Bar,Aquarium,Art Gallery,Fast Food Restaurant,Burger Joint,Caribbean Restaurant


In [33]:
atl_merged.loc[atl_merged['Cluster Labels'] == 3, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Grant Park peoplestown / Lakewood,Mexican Restaurant,Sandwich Place,Coffee Shop,Chinese Restaurant,Trail,Gas Station,Japanese Restaurant,Baseball Field,Seafood Restaurant,Smoke Shop


In [34]:
atl_merged.loc[atl_merged['Cluster Labels'] == 4, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Buckhead / North Atlanta,Grocery Store,Mexican Restaurant,Gas Station,American Restaurant,Jewelry Store,Italian Restaurant,Bagel Shop,Salon / Barbershop,Kids Store,Smoothie Shop


After analyzing this data, I focused on cluster 0.  Anecdotally, Chamblee is a racially-diverse neighborhood, while North Decatur is in the vicinity of Emory University, an institution that attracts students from around the world - particularly Asians.  I find it surprising that, with this in mind, there isn't an Asian Restaurant in the top 10 or 20 venue categories in North Decatur, while Chamblee, a zip code in which Asians rank as the 4th largest ethnic group, has 4 types of Asian restaurants in the top 10 of most prevalent venues.  To verify this hypothesis, I decided to further cluster the neighborhoods in cluster 0 to determine if, within this group, I can further refine my findings and offer a more confident recommendation.

In [24]:
atl_clust_zero = atl_merged.loc[atl_merged['Cluster Labels'] == 0, atl_merged.columns[[1] + list(range(5, atl_merged.shape[1]))]]
atl_clust_zero

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Clarkston,Discount Store,Grocery Store,Sandwich Place,Fruit & Vegetable Store,Chinese Restaurant,Storage Facility,Kids Store,Seafood Restaurant,Shoe Store,Fried Chicken Joint
3,East Lake / South Decatur,Discount Store,Wings Joint,Fast Food Restaurant,Grocery Store,Pizza Place,Gas Station,Seafood Restaurant,Fried Chicken Joint,Food,Video Store
4,North Decatur,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Fried Chicken Joint,Pizza Place,Liquor Store,Salon / Barbershop
6,Tucker,Fast Food Restaurant,Wings Joint,Mexican Restaurant,Pizza Place,Burger Joint,Southern / Soul Food Restaurant,Ice Cream Shop,Bank,Pharmacy,Pet Store
13,Adair Park / Capitol View / Oakland City / We...,Gas Station,Vegetarian / Vegan Restaurant,Park,Discount Store,Fried Chicken Joint,Pizza Place,Sandwich Place,Trail,Cosmetics Shop,Bank
14,Cascade,Park,Gym / Fitness Center,Chinese Restaurant,Caribbean Restaurant,BBQ Joint,Golf Course,Seafood Restaurant,Nature Preserve,Sandwich Place,Discount Store
17,Vines City / Mozeley Park,Gas Station,Park,Trail,Southern / Soul Food Restaurant,Pizza Place,Wings Joint,Caribbean Restaurant,Nightclub,Discount Store,Sandwich Place
19,Cabbagetown / East Atlanta Village / Ormewoo...,Bar,Discount Store,Gas Station,Park,Pizza Place,Coffee Shop,Fast Food Restaurant,Video Store,Bank,Sushi Restaurant
29,Northlake / Tucker,Fast Food Restaurant,Korean Restaurant,Sandwich Place,Sushi Restaurant,Fried Chicken Joint,Chinese Restaurant,Massage Studio,Liquor Store,Mobile Phone Shop,Bank
30,Chamblee,Chinese Restaurant,Mexican Restaurant,Asian Restaurant,Ice Cream Shop,Gym,Furniture / Home Store,Fried Chicken Joint,Vietnamese Restaurant,Korean Restaurant,Sandwich Place


In [33]:

atl_grouped_zero = atl_grouped_zero.drop(['1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue'], axis =1)
atl_grouped_zero

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,City,Clothing Store,Cocktail Bar,Coffee Shop,College & University,College Basketball Court,College Bookstore,College Rec Center,College Theater,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piercing Parlor,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Taxi,Taxi Stand,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai R

Now that I've created a normalized data set with the neighborhoods in cluster 0, I'm going to do a k means clustering exercise with only 3 clusters to ensure I get meaningful groups.

In [34]:
kclusters1 = 3

atl_grouped_clustering_zero = atl_grouped_zero.drop('Neighborhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters1, random_state=0).fit(atl_grouped_clustering_zero)

# check cluster labels generated for each row in the dataframe
kmeans1.labels_[0:10] 

array([0, 0, 1, 2, 0, 0, 2, 2, 2, 2], dtype=int32)

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cluster_zero_venues_sorted = pd.DataFrame(columns=columns)
cluster_zero_venues_sorted['Neighborhood'] = atl_grouped_zero['Neighborhood']

for ind in np.arange(atl_grouped_zero.shape[0]):
    cluster_zero_venues_sorted.iloc[ind, 1:] = return_most_common_venues(atl_grouped_zero.iloc[ind, :], num_top_venues)

cluster_zero_venues_sorted



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adair Park / Capitol View / Oakland City / We...,Gas Station,Vegetarian / Vegan Restaurant,Park,Discount Store,Fried Chicken Joint,Pizza Place,Sandwich Place,Trail,Cosmetics Shop,Bank
1,Cabbagetown / East Atlanta Village / Ormewoo...,Bar,Discount Store,Gas Station,Park,Pizza Place,Coffee Shop,Fast Food Restaurant,Video Store,Bank,Sushi Restaurant
2,Cascade,Park,Gym / Fitness Center,Chinese Restaurant,Caribbean Restaurant,BBQ Joint,Golf Course,Seafood Restaurant,Nature Preserve,Sandwich Place,Discount Store
3,Chamblee,Chinese Restaurant,Mexican Restaurant,Asian Restaurant,Ice Cream Shop,Gym,Furniture / Home Store,Fried Chicken Joint,Vietnamese Restaurant,Korean Restaurant,Sandwich Place
4,Clarkston,Discount Store,Grocery Store,Sandwich Place,Fruit & Vegetable Store,Chinese Restaurant,Storage Facility,Kids Store,Seafood Restaurant,Shoe Store,Fried Chicken Joint
5,East Lake / South Decatur,Discount Store,Wings Joint,Fast Food Restaurant,Grocery Store,Pizza Place,Gas Station,Seafood Restaurant,Fried Chicken Joint,Food,Video Store
6,East Point,Gas Station,Pharmacy,Grocery Store,Pizza Place,Breakfast Spot,Burger Joint,Fast Food Restaurant,Sandwich Place,Taco Place,Wings Joint
7,Hapeville,Mexican Restaurant,Pizza Place,Hotel,Sandwich Place,Breakfast Spot,Restaurant,Fast Food Restaurant,Liquor Store,Coffee Shop,Café
8,North Decatur,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Fried Chicken Joint,Pizza Place,Liquor Store,Salon / Barbershop
9,Northlake / Tucker,Fast Food Restaurant,Korean Restaurant,Sandwich Place,Sushi Restaurant,Fried Chicken Joint,Chinese Restaurant,Massage Studio,Liquor Store,Mobile Phone Shop,Bank


Now that I can confirm that the new analysis yields the same output as the initial, I added zip code and lat/long information to prepare the data for the mapping exercise.

In [45]:


atl_merged_zero = atlanta


atl_merged_zero = pd.merge(left = atl_merged_zero, right = cluster_zero_venues_sorted, left_on='Neighborhood', right_on='Neighborhood')
atl_merged_zero

,Zip,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,30021,Clarkston,33.809604,-84.23870,0,Discount Store,Grocery Store,Sandwich Place,Fruit & Vegetable Store,Chinese Restaurant,Storage Facility,Kids Store,Seafood Restaurant,Shoe Store,Fried Chicken Joint
1,30032,East Lake / South Decatur,33.740055,-84.26791,0,Discount Store,Wings Joint,Fast Food Restaurant,Grocery Store,Pizza Place,Gas Station,Seafood Restaurant,Fried Chicken Joint,Food,Video Store
2,30033,North Decatur,33.811354,-84.28335,2,Indian Restaurant,Fast Food Restaurant,Cosmetics Shop,Grocery Store,Discount Store,American Restaurant,Fried Chicken Joint,Pizza Place,Liquor Store,Salon / Barbershop
3,30084,Tucker,33.853853,-84.22036,2,Fast Food Restaurant,Wings Joint,Mexican Restaurant,Pizza Place,Burger Joint,Southern / Soul Food Restaurant,Ice Cream Shop,Bank,Pharmacy,Pet Store
4,30310,Adair Park / Capitol View / Oakland City / We...,33.727807,-84.41983,0,Gas Station,Vegetarian / Vegan Restaurant,Park,Discount Store,Fried Chicken Joint,Pizza Place,Sandwich Place,Trail,Cosmetics Shop,Bank
5,30311,Cascade,33.724907,-84.47131,1,Park,Gym / Fitness Center,Chinese Restaurant,Caribbean Restaurant,BBQ Joint,Golf Course,Seafood Restaurant,Nature Preserve,Sandwich Place,Discount Store
6,30314,Vines City / Mozeley Park,33.758056,-84.42268,0,Gas Station,Park,Trail,Southern / Soul Food Restaurant,Pizza Place,Wings Joint,Caribbean Restaurant,Nightclub,Discount Store,Sandwich Place
7,30316,Cabbagetown / East Atlanta Village / Ormewoo...,33.729506,-84.34087,0,Bar,Discount Store,Gas Station,Park,Pizza Place,Coffee Shop,Fast Food Restaurant,Video Store,Bank,Sushi Restaurant
8,30340,Northlake / Tucker,33.899381,-84.25367,2,Fast Food Restaurant,Korean Restaurant,Sandwich Place,Sushi Restaurant,Fried Chicken Joint,Chinese Restaurant,Massage Studio,Liquor Store,Mobile Phone Shop,Bank
9,30341,Chamblee,33.888102,-84.29321,2,Chinese Restaurant,Mexican Restaurant,Asian Restaurant,Ice Cream Shop,Gym,Furniture / Home Store,Fried Chicken Joint,Vietnamese Restaurant,Korean Restaurant,Sandwich Place


In [47]:
# create map
map_cluster_zero = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters1)
ys = [i + x + (i*x)**2 for i in range(kclusters1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(atl_merged_zero['Latitude'], atl_merged_zero['Longitude'], atl_merged_zero['Neighborhood'], atl_merged_zero['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster_zero)
       
map_cluster_zero

The more detailed analysis shows that, indeed, Chamblee and North Decatur are in the same microcluster and, thus, that we can support our hypothesis with data.   